# Assignment 7: Semantic Segmentation
Heute werden wir einfache Netzwerkarchitekturen für "Semantic Segmentation" testen. Ziel ist es dieses Paper in den Grundzügen zu implementieren: https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf. Bitte lesen!

## Daten

Es gibt einige gute Datensätze, die ihr (bei gegebener Hardware) herunterladen und benutzen könnt. Für diejenigen, die auf CPUs rechnen gilt immer der Tip: Bilder downsamplen!

Sucht Euch einen Satensatz aus: 

KITTI: http://www.cvlibs.net/download.php?file=data_semantics.zip (~300 MB, 200 Bilder)

DUS: http://www.6d-vision.com/scene-labeling (~3 GB, 500 Bilder)

MIT. http://sceneparsing.csail.mit.edu/ (~1 GB, links siehe auf Seite)

## Exc. 7.1 Fully convolutional network, no downsampling
Implementiere die in der Vorlesung besprochene Netzwerkarchitektur von aufeinanderfolgenden CONV-Schichten (stride=1, mit zero-padding), um eine Ausgabeschicht zu bekommen, die die Eingabegröße aufweist. Tip: die letzte CONV-Schicht sollte eine Tiefe haben, die zur Zahl der Klassen korrespondiert. Benutze den L2-Loss zum Labelbild (Achtung: ihr müsst dafür entweder das Labelbild oder den Ausgabetensor umformulieren).

Trainiere das Netzwerk auf den von Dir gewählten Datensatz und zeige den Verlauf des Losses, und einige zufällig gewählte Beispielbilder mit ihren vorhergesagten Segmentierungen an. (**RESULT**)

In [76]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

image_generator = ImageDataGenerator()
label_generator = ImageDataGenerator()

# Beispiel für den KITTI-Datensatz. Ich habe die 200 training samples in 180 train- und 20 testbilder
# geteilt (macht 180 samples inkl. labels)
# um uns das Leben leichter zu machen, Bilder heruntersamplen
img_size = (38, 125)


# Bild- und Label-Generator
Q1 = image_generator.flow_from_directory( 'data_semantics/training/image_2',
                                        class_mode=None,
                                        batch_size=1,
                                        target_size=img_size, seed=1)

Q2 = label_generator.flow_from_directory( 'data_semantics/training/semantic_rgb',
                                        class_mode=None,
                                        batch_size=1,
                                        target_size=img_size, seed=1)

# ... kombinieren
train_generator = zip(Q1, Q2)

# mach der Definition des Modells:
# model.fit_generator( train_generator, steps_per_epoch = 1000, epochs = 100)


Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.


In [77]:
from keras.models import Sequential
from keras.layers import Conv2D, Dropout
from keras.optimizers import SGD
from keras import regularizers

In [78]:
model = Sequential([
    Conv2D(4, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01), input_shape=img_size + (3,)),
    Dropout(0.4),
    
    Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.4),

    Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.4),
    
    Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.4),

    Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.4),

    Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.4),
    
    Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.4),
    
    Conv2D(4, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.4),
    
    Conv2D(3, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))
])

In [79]:
model.compile(optimizer=SGD(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [80]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_144 (Conv2D)          (None, 38, 125, 4)        112       
_________________________________________________________________
dropout_128 (Dropout)        (None, 38, 125, 4)        0         
_________________________________________________________________
conv2d_145 (Conv2D)          (None, 38, 125, 8)        296       
_________________________________________________________________
dropout_129 (Dropout)        (None, 38, 125, 8)        0         
_________________________________________________________________
conv2d_146 (Conv2D)          (None, 38, 125, 8)        584       
_________________________________________________________________
dropout_130 (Dropout)        (None, 38, 125, 8)        0         
_________________________________________________________________
conv2d_147 (Conv2D)          (None, 38, 125, 16)       1168      
__________

In [82]:
model.fit_generator(train_generator, steps_per_epoch = 250, epochs = 50)

Epoch 1/50
250/250 [==============================] - 11s 45ms/step - loss: nan - acc: 0.4258
Epoch 2/50
250/250 [==============================] - 13s 50ms/step - loss: nan - acc: 0.4185
Epoch 3/50
250/250 [==============================] - 12s 48ms/step - loss: nan - acc: 0.4216
Epoch 4/50
250/250 [==============================] - 11s 45ms/step - loss: nan - acc: 0.4200
Epoch 5/50
250/250 [==============================] - 11s 45ms/step - loss: nan - acc: 0.4138
Epoch 6/50
250/250 [==============================] - 11s 46ms/step - loss: nan - acc: 0.4316
Epoch 7/50
250/250 [==============================] - 11s 46ms/step - loss: nan - acc: 0.4116
Epoch 8/50
250/250 [==============================] - 11s 44ms/step - loss: nan - acc: 0.4225
Epoch 9/50
250/250 [==============================] - 11s 45ms/step - loss: nan - acc: 0.4204
Epoch 10/50
250/250 [==============================] - 11s 45ms/step - loss: nan - acc: 0.4155
Epoch 11/50
250/250 [==============================] - 11s 

Uns ist nicht ganz klar, warum kein loss angezeigt wird. Eventuell liegt es an der Ordnerstruktur der Daten, die wir angepasst haben, damit die Generator überhaupt die Bilder einlesen.

# Exc. 7.2 FCN mit Bottleneck

Implementiere jetzt die Variante mit schrittweisem Down- und Upsampling, wie in der Vorlesung besprochen. Nutze dafür ein bestehendes Netzwerk (z.B. VGG16, https://keras.io/applications/#vgg16), entferne die FC-Schichten am Ende, und füge dann die Upsampling-Schichten hinzu. Wie in der vorigen Vorlesung zu Transfer Learning beschrieben, kannst Du jetzt nur den zweiten Teil trainieren und die Gewichte des ersten Teils "einfrieren".

Stelle wie oben den Verlauf des Losses dar und wähle einige Beispielbilder aus dem Testset und zeige sie mit ihrer vorhergesagten Segmentierung an. (**BONUS**)